In [1]:
cd ..

/Users/amlvt225/Code/GitHub/donkey_golf


In [2]:
from inspect import getsource

import donkey_golf as dg

import donkey_golf.data_things as DT

import oyaml
import pandas as pd
import logging
import psycopg2

from sqlalchemy import create_engine

from donkey_golf import config

# create logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

/Users/amlvt225/Code/VirtualEnvs/venv_dgolf/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [3]:
gs = DT.GameScoreboard()

2019-05-14 22:17:21,163 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-05-14 22:17:21,172 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-14 22:17:21,444 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056552
2019-05-14 22:17:21,445 - donkey_golf.data_things.base_class - INFO - Executing run_sql


In [7]:
gs.run()

2019-05-14 22:18:21,841 - donkey_golf.data_things.game_scoreboard - INFO - Not pulling any scoreboard data since its pre_tourney
2019-05-14 22:18:21,843 - donkey_golf.data_things.game_scoreboard - INFO - Pulling pre tourney draft info
2019-05-14 22:18:21,844 - donkey_golf.data_things.base_class - INFO - Executing run_sql


In [8]:
gs.df_drafted_teams

,id,username,pay_flag
0,12,Jeff,None
1,17,Ryan Tarkington,None
2,16,Tiger's Back,None
3,11,fleet,None
4,1,cloud,yes
5,10,ocean,yes
6,4,mp,None
7,3,chubbs,None
8,18,Billsfan21104,None


In [ ]:
lb = DT.PullLeaderboard(user_id=6)

In [ ]:
lb.tourney_status

In [ ]:
lb.run()

In [ ]:
lb.pull_pre_tourney_data()

In [ ]:
lb.df_pre_tourney['current_rank'] = lb.df_pre_tourney['current_rank'].fillna(9999).astype(int).astype(str)

lb.df_pre_tourney['current_rank'] = lb.df_pre_tourney['current_rank'].replace('9999', '-')

In [ ]:
lb.df_pre_tourney

In [ ]:
dd = DT.DraftDay()

In [ ]:
dd.run()

In [ ]:
df = dd.data

In [ ]:
# top 8 pick 2
# next 12 pick 3
# 

In [ ]:
df

In [ ]:
df['rank'] = df['rank'].fillna(999)
df['rank'] = df['rank'].astype(int)
df['tier'] = 'Tier 3'
df.loc[df['rank'] <= 8, 'tier'] = 'Tier 1'
df.loc[(df['rank'] >= 9) & (df['rank'] <= 16), 'tier'] = 'Tier 2'

In [ ]:
df

In [ ]:
lb.df_pre_tourney.loc[lb.df_pre_tourney['current_rank'].notnull(), 'current_rank'] = lb.df_pre_tourney.loc[lb.df_pre_tourney['current_rank'].notnull(), 'current_rank'].astype(int)

In [ ]:
lb.df_pre_tourney.loc[lb.df_pre_tourney['current_rank'].notnull(), 'current_rank'].astype(int)

In [ ]:
lb.df_pre_tourney

In [ ]:
DT.LoadWorldRankings().run()

In [ ]:
DT.TournamentInfo().run()

In [ ]:
lb.run()

In [ ]:
lb.data

In [ ]:
lb.determine_current_round()

In [ ]:
lb.calculate_cut_line()

In [ ]:
df = lb.data

In [ ]:
self = lb

In [ ]:
cur_round = self.determine_current_round()

if cur_round == 'r2':
    sql = self.yaml_sql_dict.get('calculate_cut_line')
    df = self.run_sql(sql)
    # Grab the cut_str
    cut_str = df['pos'][0].lower()
    cut_dict = {}

    # Not sure what will happen if cut is even...
    if cut_str[-1:] == 'E':
        cut_dict['cut_str'] = 'E'
        cut_dict['cut_in'] = 0
    else:
        # Just grab the last two characters - this should be the cut score
        cut_str = cut_str[-2:]
        # Put both string and int representations into a dict
        cut_dict['cut_str'] = cut_str
        cut_dict['cut_int'] = int(cut_str.replace('+', '').replace('E', '0'))

    return cut_dict
else:
    logger.info("Dont need to worry about the cut line now")
    return None

In [ ]:
df

In [ ]:
print(sql)

In [ ]:
sql = self.yaml_sql_dict.get('calculate_cut_line')
df = self.run_sql(sql)
# Grab the cut_str
cut_str = df['pos'][0].lower()
cut_dict = {}

In [ ]:
user = DT.UserData(user_id=6)

In [ ]:
user.run()

In [ ]:
user.data

In [ ]:
gs = DT.GameScoreboard()

In [ ]:
gs.run()

In [ ]:
gs.data

In [ ]:
wha = DT.TeamPlayerResults()

In [ ]:
wha.run()

In [ ]:
wha.data.sort_values('username')

In [ ]:
lb = DT.LoadLeaderboard()

In [ ]:
lb.run()

In [ ]:
gs = dg.data_things.GameScoreboard()

In [ ]:
int(0)

In [ ]:
gs.tourney_status

In [ ]:
gs.run()

In [ ]:
cmon = dg.data_things.LoadLeaderboard()

In [ ]:
cmon.run()

In [ ]:
lb = dg.data_things.LoadLeaderboard()

In [ ]:
lb.tourney_status

In [ ]:
lb.scrape_espn_leaderboard(lb.url_lb)

In [ ]:
lb.data

In [ ]:
lb.determine_tourney_status()

In [ ]:
lb.load_tourney_status()

In [ ]:
vamp.run()

In [ ]:
vamp.tourney_status

In [ ]:
new = dg.data_things.PullLeaderboard(user_id=10)

In [ ]:
new.tourney_status

In [ ]:
new.run()

In [ ]:
new.pull_tourney_leaderboard()

In [ ]:
new.user_df

In [ ]:
new.user_id

In [ ]:
new.pull_users_team()

In [ ]:
new.data

In [ ]:
new.pull_tourney_leaderboard()

In [ ]:
getattr(new, 'user_df')

In [ ]:
print(getsource(new.pull_tourney_leaderboard))

In [ ]:
new.user_df

In [ ]:
pd.merge(new.data, new.user_df,
        how='outer',
        on='player')

In [ ]:
new.tourney_status

In [ ]:
gs = dg.data_things.GameScoreboard()

In [ ]:
gs.tourney_status

In [ ]:
gs.run()

In [ ]:
wha = dg.data_things.LoadWorldRankings()

In [ ]:
wha = dg.data_things.TournamentInfo()

In [ ]:
wha.run()

In [ ]:
draft = dg.data_things.DraftDay()

In [ ]:
draft.run()

In [ ]:
df = draft.data

In [ ]:
for col in ['current_rank','events_played','rank']:
    self.data.loc[self.data[col].notnull(), col] = self.data.loc[self.data[col].notnull(), col].astype(int).astype(str)

In [ ]:
df

In [ ]:
draft.data['current_rank']

In [ ]:
vamp.new_determine_status()['value'][0].strip()

In [ ]:
vamp.run()

In [ ]:
vamp.load_tourney_status()

In [ ]:
vamp.load_status_sql

In [ ]:
with psycopg2.connect(vamp.conf.db_url) as conn:
    cur = conn.cursor()
    cur.execute("delete from donkey_variables where metric = 'current_tourney_status';")
    cur.execute("INSERT INTO donkey_variables VALUES ('current_tourney_status', 'pre_tourney');")
    #cur.close()

In [ ]:
conn = psycopg2.connect(host=pg_credential.hostname,
                        port=pg_credential.port,
                        user=pg_credential.username,
                        password=pg_credential.password,
                        database=pg_credential.path[1:]) # To remove slash

cursor = conn.cursor()
cursor.execute("INSERT INTO a_table (c1, c2, c3) VALUES(%s, %s, %s)", (v1, v2, v3))

In [ ]:
getattr(cmon, 'tourney_status')

In [ ]:
vamp.load_tourney_status()

In [ ]:
vamp.tourney_status

In [ ]:
pd.read_sql("INSERT INTO donkey_variables VALUES ('current_tourney_status', 'pre_tourney');")

In [ ]:
cmon = dg.data_things.PullLeaderboard(user_id=5)

In [ ]:
cmon.run()

In [ ]:
cmon.tourney_status

In [ ]:
cmon.pull_tourney_leaderboard()

In [ ]:
if cmon.data.empty:
    print("EMPTY")

In [ ]:
cmon.tourney_status

In [ ]:
getattr(cmon, 'tourney_status')

In [ ]:
getattr(cmon, 'tourney_statusss', 'whoops')

In [ ]:
sb = dg.data_things.GameScoreboard()

In [ ]:
sb.run()

In [ ]:
vamp.golfer_count_by_team

In [ ]:
df = vamp.data

In [ ]:
df.head(2)

In [ ]:
df.loc[df['team_count']> 0]

In [ ]:
huh = dg.data_things.PullLeaderboard(user_id=6)

In [ ]:
huh.run()

In [ ]:
huh.data['team_count'].value_counts()

In [ ]:
df_tourney = huh.data
df_tourney = df_tourney.loc[df_tourney['team_count'] > 0]

In [ ]:
from datetime import datetime

In [ ]:
datetime.today()

In [ ]:
df_tourney

In [ ]:
ada = dg.data_things.PullLeaderboard(user_id=5)

In [ ]:
ada.pull_tourney_leaderboard()

In [ ]:
ada.data.team_count.value_counts()

In [ ]:
ada.calc_refresh_date()

In [ ]:
ada.determine_tourney_status()

In [ ]:
ada.pull_users_team()

In [ ]:
ada.user_df

In [ ]:
ada.sql_user